<a href="https://colab.research.google.com/github/sathbt/Finetuning-LLM/blob/main/llamaindex/Combining_Text_to_SQL_with_Semantic_Search_for_Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combining Text-to-SQL with Semantic Search for `RAG` [LlamaIndex Website](https://www.llamaindex.ai/)

In this tutorial, we show you how to use our `SQLAutoVectorQueryEngine`. More info in this [blog post](https://blog.llamaindex.ai/combining-text-to-sql-with-semantic-search-for-retrieval-augmented-generation-c60af30ec3b).

- This query engine allows you to combine insights from your structured tables with your unstructured data.
- Can leverage both a SQL database as well as a vector store to fulfill complex natural language queries over a combination of structured and unstructured data
- It first decides whether to query your structured tables for insights. Once it does, it can then infer a corresponding query to the vector store in order to fetch corresponding documents.



## SETUP

In [1]:
%%capture
!pip install llama-index openai pinecone-client

In [2]:
import openai
import os

# find API key in console at https://platform.openai.com/account/api-keys

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SQLDatabase,
    WikipediaReader,
)

ImportError: cannot import name 'VectorStoreIndex' from 'llama_index' (unknown location)

## Create Common Objects
This includes a `ServiceContext` object containing abstractions such as the LLM and chunk size. This also includes a `StorageContext` object containing our vector store abstractions.

In [ ]:
# define pinecone index
import pinecone
import os

# find API key in console at https://app.pinecone.io/
os.environ['PINECONE_API_KEY'] = 'PINECONE_API_KEY'
# environment is found next to API key in the console
os.environ['PINECONE_ENVIRONMENT'] = 'asia-southeast1-gcp'

# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

# dimensions are for text-embedding-ada-002
pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

In [ ]:
# list indexes
pinecone.list_indexes()

In [ ]:
# describe index
pinecone.describe_index("quickstart")

In [ ]:
# connect to the index
pinecone_index = pinecone.Index('quickstart')

In [ ]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI

# define node parser and LLM
chunk_size = 1024
llm = OpenAI(temperature=0, model="gpt-3.5-turbo", streaming=True)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser.from_defaults(text_splitter=text_splitter)

# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wiki_cities"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

## Create Database Schema + Test Data

In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [ ]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [ ]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [ ]:
# print tables
metadata_obj.tables.keys()

We introduce some test data into the `city_stats` table

In [ ]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

## Load Data
- Lets use the [Wikipedia Loader](https://llamahub.ai/l/wikipedia) from LlamaHub.

In [ ]:
%%capture
!pip install wikipedia

In [ ]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()
cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = loader.load_data(pages=cities)

In [ ]:
len(wiki_docs)

## Build SQL Index

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [ ]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [ ]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

## Build Vector Index

In [ ]:
# Insert documents into vector index
# Each document has metadata of the city attached
for city, wiki_doc in zip(cities, wiki_docs):
    nodes = node_parser.get_nodes_from_documents([wiki_doc])
    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": city}
    vector_index.insert_nodes(nodes)

## Define Query Engines, Set as Tools

In [ ]:
from llama_index.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.indices.vector_store import VectorIndexAutoRetriever
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo

In [ ]:
vector_store_info = VectorStoreInfo(
    content_info="articles about different cities",
    metadata_info=[
        MetadataInfo(name="title", type="str", description="The name of the city"),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, service_context=service_context
)

In [ ]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "city_stats, containing the population/country of each city"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=f"Useful for answering semantic questions about different cities",
)

## Define `SQLAutoVectorQueryEngine`

In [ ]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, service_context=service_context
)

## Query
**The original question, SQL query, SQL response, vector store query, and vector store response are combined into a prompt to synthesize the final answer.**

In [ ]:
response = query_engine.query("Can you give me the country corresponding to each city?")
response.response

In [ ]:
response.metadata

In [ ]:
response = query_engine.query("Tell me about the history of Berlin")
response.response

In [ ]:
response.metadata

In [ ]:
response = query_engine.query(
    "Tell me about the arts and culture of the city with the highest population"
)

In [ ]:
response.response

### GPT-4 is needed for Querying both tool

In [ ]:
llm = OpenAI(temperature=0.7, model="gpt-4", streaming=True)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)

query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, service_context=service_context
)

In [ ]:
response = query_engine.query(
    "Tell me about the arts and culture of the city with the highest population"
)

In [ ]:
response.response

In [ ]:
response.metadata

In [ ]:
# delete the pinecone index
pinecone.delete_index("quickstart")

## Conclusion
- Chunking plays a vital role.
- Need to play around with different models to get the right answer.
- Trial and error is what it is needed for LLMs.